In [2]:
from pathlib import Path
from openai import OpenAI
import json
from datetime import datetime
TeleQnA = "C:/Users/wasd0/Desktop/AMD-AI-Agent-Online-Hackthon/branchmark/TeleQnA/TeleQnA.txt"


In [3]:
with open(TeleQnA, "r", encoding="utf-8") as f:
    teleqna_data = json.load(f)

# 顯示一筆資料範例
sample_key = list(teleqna_data.keys())[0]
teleqna_data[sample_key]


{'question': 'What is the purpose of the Nmfaf_3daDataManagement_Deconfigure service operation? [3GPP Release 18]',
 'option 1': 'To configure the MFAF to map data or analytics received by the MFAF to out-bound notification endpoints',
 'option 2': 'To configure the MFAF to stop mapping data or analytics received by the MFAF to out-bound notification endpoints',
 'option 3': 'To supply data or analytics from the MFAF to notification endpoints',
 'option 4': 'To fetch data or analytics from the MFAF based on fetch instructions',
 'answer': 'option 2: To configure the MFAF to stop mapping data or analytics received by the MFAF to out-bound notification endpoints',
 'explanation': 'The Nmfaf_3daDataManagement_Deconfigure service operation is used to stop mapping data or analytics received by the MFAF to one or more out-bound notification endpoints.',
 'category': 'Standards specifications'}

In [8]:
def call_nvidia_llm(prompt: str) -> str:
    client = OpenAI(
        base_url="https://integrate.api.nvidia.com/v1",
        api_key = "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"
    )

    response = client.chat.completions.create(
        model="microsoft/phi-4-mini-flash-reasoning",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.5,
        top_p=0.7,
        max_tokens=1024,
        stream=False
    )

    return response.choices[0].message.content

# 📤 生成 prompt 並取得回應
def generate_prompt(entry: dict) -> str:
    prompt = f"""You are a telecom expert. Please answer the following multiple choice question with an explanation.

Question: {entry['question']}

Options:
(A) {entry['option 1']}
(B) {entry['option 2']}
(C) {entry['option 3']}
(D) {entry['option 4']}

Please provide:
1. The correct option (A/B/C/D)
2. A brief explanation.
"""
    return prompt

In [9]:
# 🔄 測試一筆資料
entry = teleqna_data["question 2"]
prompt = generate_prompt(entry)
llm_response = call_nvidia_llm(prompt)
print("LLM 回應內容:\n", llm_response)

LLM 回應內容:
 <think>['1. **Identify the core concept:** The question is about the Alamouti scheme and its diversity gain.\n\n2. **Recall knowledge about Alamouti scheme:**  The Alamouti scheme is a direct detection method used in MDS codes. Its primary goal is *to simplify detection* of symbols, particularly in the presence of amplitude modulation.\n\n3. **Connect "simplify detection" to diversity gain:**  A diversity gain in detection means the ability to distinguish between multiple signals or symbols. While the Alamouti scheme performs detection, the *diversity gain* isn\'t its main purpose.\n\n4. **Eliminate incorrect options:**\n    * (A) 0: A diversity gain of 0 means no ability to distinguish, which contradicts the purpose of the Alamouti scheme.\n    * (D) 1: While it might seem like the basic detection capability is one unit, the key aspect is *improvement* in detection due to diversity.\n\n5. **Focus on the "simplicity" aspect:** The Alamouti scheme simplifies detection. This s

In [10]:
# 🔍 比對 LLM 回答與正解
from difflib import SequenceMatcher

def match_option(llm_output: str, correct_answer: str) -> dict:
    correct_option = correct_answer.split(":")[0].strip().split()[-1]  # option 3 → 3
    correct_letter = chr(ord("A") + int(correct_option) - 1)  # 3 → C

    llm_guess = "?"
    for letter in ["A", "B", "C", "D"]:
        if f"({letter})" in llm_output or f"Option {letter}" in llm_output or f"Answer: {letter}" in llm_output:
            llm_guess = letter
            break

    is_correct = (llm_guess == correct_letter)

    return {
        "llm_guess": llm_guess,
        "correct_letter": correct_letter,
        "is_correct": is_correct,
        "llm_explanation": llm_output
    }

match_result = match_option(llm_response, entry["answer"])
match_result

{'llm_guess': 'A',
 'correct_letter': 'C',
 'is_correct': False,
 'llm_explanation': '<think>[\'1. **Identify the core concept:** The question is about the Alamouti scheme and its diversity gain.\\n\\n2. **Recall knowledge about Alamouti scheme:**  The Alamouti scheme is a direct detection method used in MDS codes. Its primary goal is *to simplify detection* of symbols, particularly in the presence of amplitude modulation.\\n\\n3. **Connect "simplify detection" to diversity gain:**  A diversity gain in detection means the ability to distinguish between multiple signals or symbols. While the Alamouti scheme performs detection, the *diversity gain* isn\\\'t its main purpose.\\n\\n4. **Eliminate incorrect options:**\\n    * (A) 0: A diversity gain of 0 means no ability to distinguish, which contradicts the purpose of the Alamouti scheme.\\n    * (D) 1: While it might seem like the basic detection capability is one unit, the key aspect is *improvement* in detection due to diversity.\\n\\n5

In [11]:
import time
from openai import OpenAI

def evaluate_model_on_teleqna(model_name: str, num_questions: int = 10, delay_sec: int = 2):
    print(f"\n🚀 Testing model: {model_name}")
    client = OpenAI(
        base_url="https://integrate.api.nvidia.com/v1",
        api_key="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"
    )

    correct_count = 0
    results = []
    all_keys = list(teleqna_data.keys())[:num_questions]

    for idx, key in enumerate(all_keys, start=1):
        entry = teleqna_data[key]
        prompt = generate_prompt(entry)
        try:
            response = client.chat.completions.create(
                model=model_name,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.5,
                top_p=0.7,
                max_tokens=1024,
                stream=False
            )
            llm_response = response.choices[0].message.content

            result = match_option(llm_response, entry["answer"])
            result["question_key"] = key
            results.append(result)

            if result["is_correct"]:
                correct_count += 1

            current_accuracy = correct_count / idx
            status = "✅ Correct" if result["is_correct"] else "❌ Wrong"
            print(f"[{idx:02}] {status} | Guess: {result['llm_guess']} | Ans: {result['correct_letter']} | Acc: {current_accuracy:.2%}")
        except Exception as e:
            print(f"[{idx:02}] ⚠️ Error in question {key}: {e}")
            results.append({"question_key": key, "error": str(e)})

        time.sleep(delay_sec)

    final_accuracy = correct_count / len(all_keys)
    print(f"🎯 Final Accuracy for {model_name}: {final_accuracy:.2%}")
    return {
        "model": model_name,
        "accuracy": final_accuracy,
        "results": results
    }

In [14]:
candidate_models = [
# "meta/llama-3.1-405b-instruct",
# "meta/llama-3.1-70b-instruct",
"meta/llama-3.1-8b-instruct"
]

start_time = datetime.now()
none_rag_results = []
for model in candidate_models:
    result = evaluate_model_on_teleqna(model_name=model, num_questions=20, delay_sec=0.5)
    none_rag_results.append(result)


data_to_save = none_rag_results

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
json_path = Path(f"C:/Users/wasd0/Desktop/AMD-AI-Agent-Online-Hackthon/none-RAG_evaluation_results_{timestamp}.json")
end_time = datetime.now()
time_taken = end_time - start_time
print(f"Time taken for evaluation: {time_taken}")



with open(json_path, "w", encoding="utf-8") as f:
    json.dump(data_to_save, f, ensure_ascii=False, indent=2)

print(f"✅ Saved to: {json_path}")


🚀 Testing model: meta/llama-3.1-8b-instruct
[01] ✅ Correct | Guess: B | Ans: B | Acc: 100.00%
[02] ❌ Wrong | Guess: D | Ans: E | Acc: 50.00%
[03] ✅ Correct | Guess: C | Ans: C | Acc: 66.67%
[04] ❌ Wrong | Guess: A | Ans: D | Acc: 50.00%
[05] ❌ Wrong | Guess: A | Ans: B | Acc: 40.00%
[06] ✅ Correct | Guess: C | Ans: C | Acc: 50.00%
[07] ✅ Correct | Guess: B | Ans: B | Acc: 57.14%
[08] ✅ Correct | Guess: A | Ans: A | Acc: 62.50%
[09] ✅ Correct | Guess: A | Ans: A | Acc: 66.67%
[10] ✅ Correct | Guess: B | Ans: B | Acc: 70.00%
[11] ❌ Wrong | Guess: A | Ans: C | Acc: 63.64%
[12] ✅ Correct | Guess: C | Ans: C | Acc: 66.67%
[13] ✅ Correct | Guess: D | Ans: D | Acc: 69.23%
[14] ❌ Wrong | Guess: B | Ans: A | Acc: 64.29%
[15] ❌ Wrong | Guess: A | Ans: B | Acc: 60.00%
[16] ✅ Correct | Guess: C | Ans: C | Acc: 62.50%
[17] ❌ Wrong | Guess: D | Ans: E | Acc: 58.82%
[18] ✅ Correct | Guess: C | Ans: C | Acc: 61.11%
[19] ✅ Correct | Guess: A | Ans: A | Acc: 63.16%
[20] ❌ Wrong | Guess: D | Ans: B | Ac